https://github.com/arturzeitler/Bayes-and-MC/blob/master/Bayesian_NN_Example.ipynb

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd
import pickle

In [3]:
from tensorflow_probability import edward2 as ed 

In [4]:
# Use the TensorFlow method to download and/or load the data.
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.


/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 12:13:45.877577. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [33]:
#two-layer neural network.
def neural_network(x, W_0, W_1, b_0, b_1):
    h = tf.matmul(x, W_0) + b_0
    h = tf.tanh(h)
    #h = tf.exp(tf.negative(tf.square(h)))
    h = tf.matmul(h, W_1) + b_1
    return h

def display_draws(loss, train_x, EPOCH_NUM):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16, 3.5))
    plt.plot(np.arange(EPOCH_NUM), loss / len(train_x), label='Train')
    plt.legend(fontsize=20)
    plt.xlabel('Epoch', fontsize=15)
    plt.ylabel('Negated ELBO', fontsize=15)
    plt.show()

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 16:55:40.541940. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [42]:
import copy

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:17:20.258500. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [57]:
%run optim.py

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:19:31.570139. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


TypeError: Error when calling the metaclass bases
    module.__init__() takes at most 2 arguments (3 given)

In [52]:
total = mnist.train.num_examples #number of training examples
N = total/10   # number of images in a minibatch.
D = 784   # number of features.
K = 10    # number of classes.
P = 100   # number of neurons.

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:18:53.092518. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [53]:
# Create a placeholder to hold the data (in minibatches) in a TensorFlow graph.
x = tf.placeholder(tf.float32, [None, D])
# Normal(0,1) priors for the variables. Note that the syntax assumes TensorFlow 1.1.
w0 = ed.Normal(loc=tf.zeros([D, P]), scale=tf.ones([D, P]))
b0 = ed.Normal(loc=tf.zeros(P), scale=tf.ones(P))
w1 = ed.Normal(loc=tf.zeros([P, K]), scale=tf.ones([P, K]))
b1 = ed.Normal(loc=tf.zeros(K), scale=tf.ones(K))
# Categorical likelihood for classication.
y =ed.Categorical(neural_network(x, w0,w1, b0, b1))

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:18:53.318629. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [54]:
# Contruct the q(w) and q(b). in this case we assume Normal distributions.
qw0 = ed.Normal(loc=tf.Variable(tf.random_normal([D, P])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, P]))))
qb0 = ed.Normal(loc=tf.Variable(tf.random_normal([P])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([P]))))
qw1 = ed.Normal(loc=tf.Variable(tf.random_normal([P, K])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([P, K]))))
qb1 = ed.Normal(loc=tf.Variable(tf.random_normal([K])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([K]))))

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:18:53.543986. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [55]:
epoch_num = 200
train_losssgd = np.zeros(epoch_num)

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:18:53.791490. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [56]:
# We use a placeholder for the labels in anticipation of the traning data.
y_ph = tf.placeholder(tf.int32, [N])
# Define the VI inference technique, ie. minimise the KL divergence between q and p.
inference = elbo_optimizer({w0: qw0, b0: qb0,w1:qw1,b1:qb1}, data={y:y_ph})

/Users/karimimohammedbelhal/Desktop/ongoing/hostnfly/deeptech/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-09-21 17:18:54.252851. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


TypeError: must be type, not classobj

# BayesByBackprop (SGD with MC estimate of gradients and reparam trick)

In [ ]:
for i in (0,total/N):
    X_batch, Y_batch = mnist.train.next_batch(N)
    # TensorFlow method gives the label data in a one hot vetor format. We convert that into a single label.
    Y_batch = np.argmax(Y_batch,axis=1)
    grads = tf.gradients(newloss, tf.trainable_variables())
    var_updates = []
    var_list = tf.trainable_variables()
    for grad, var in zip(grads, var_list):
        var_updates.append(var.assign_sub(learning_rate * grad))
    train_op = tf.group(*var_updates)
    sess.run(train_op,feed_dict={x: X_batch, y_ph: Y_batch})
    #info_dict = inference.update(feed_dict={x: X_batch, y_ph: Y_batch})
        
train_losssgd[0] = sess.run(newloss,feed_dict={x: X_batch, y_ph: Y_batch})

In [ ]:
for epoch in range(1,epoch_num):
    for i in (0,total/N):
        print(epoch)
        X_batch, Y_batch = mnist.train.next_batch(N)
        # TensorFlow method gives the label data in a one hot vetor format. We convert that into a single label.
        Y_batch = np.argmax(Y_batch,axis=1)
        grads = tf.gradients(newloss, tf.trainable_variables())
        var_updates = []
        var_list = tf.trainable_variables()
        for grad, var in zip(grads, var_list):
            var_updates.append(var.assign_sub(learning_rate * grad))
        train_op = tf.group(*var_updates)
        sess.run(train_op,feed_dict={x: X_batch, y_ph: Y_batch})
        #info_dict = inference.update(feed_dict={x: X_batch, y_ph: Y_batch})
        
    train_losssgd[epoch] = sess.run(newloss,feed_dict={x: X_batch, y_ph: Y_batch})

In [ ]:
for epoch in range(1,epoch_num):
    for _ in range(0, total, N):
        X_batch, Y_batch = mnist.train.next_batch(N)
        # TensorFlow method gives the label data in a one hot vetor format. We convert that into a single label.
        Y_batch = np.argmax(Y_batch,axis=1)
        info_dict = inference.update(feed_dict={x: X_batch, y_ph: Y_batch})
        
    train_losssgd[epoch] = info_dict['loss']
    inference.print_progress(info_dict)

In [ ]:
sgd = train_losssgd[0:100]
#save loss
with open('sgdloss', 'wb') as fp:
    pickle.dump(sgd, fp)

In [ ]:
display_draws(train_losssgd, X_batch, epoch_num)

# ADAM

In [ ]:
total = mnist.train.num_examples #number of training examples
N = total/10  # number of images in a minibatch.
D = 784   # number of features.
K = 10    # number of classes.
P = 100   # number of neurons.
# Create a placeholder to hold the data (in minibatches) in a TensorFlow graph.
x = tf.placeholder(tf.float32, [None, D])
# Normal(0,1) priors for the variables. Note that the syntax assumes TensorFlow 1.1.
w0 = Normal(loc=tf.zeros([D, P]), scale=tf.ones([D, P]))
b0 = Normal(loc=tf.zeros(P), scale=tf.ones(P))
w1 = Normal(loc=tf.zeros([P, K]), scale=tf.ones([P, K]))
b1 = Normal(loc=tf.zeros(K), scale=tf.ones(K))
# Categorical likelihood for classication.
y =Categorical(neural_network(x, w0,w1, b0, b1))
# Contruct the q(w) and q(b). in this case we assume Normal distributions.
qw0 = Normal(loc=tf.Variable(tf.random_normal([D, P])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([D, P]))))
qb0 = Normal(loc=tf.Variable(tf.random_normal([P])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([P]))))
qw1 = Normal(loc=tf.Variable(tf.random_normal([P, K])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([P, K]))))
qb1 = Normal(loc=tf.Variable(tf.random_normal([K])),
              scale=tf.nn.softplus(tf.Variable(tf.random_normal([K]))))
epoch_num = 200
train_lossadam = np.zeros(epoch_num)
# We use a placeholder for the labels in anticipation of the traning data.
y_ph = tf.placeholder(tf.int32, [N])
# Define the VI inference technique, ie. minimise the KL divergence between q and p.
inference = elbo_optimizer({w0: qw0, b0: qb0,w1:qw1,b1:qb1}, data={y:y_ph})

learning_rate = 5e-3
optimizer = tf.train.AdamOptimizer(learning_rate)

inference.initialize(optimizer=optimizer,n_print=100,n_samples=mcbatch)

# We will use an interactive session.
sess = tf.InteractiveSession()
# Initialise all the vairables in the session.
tf.global_variables_initializer().run()
obj = inference.build_loss_and_gradients(tf.trainable_variables())

newloss = obj[0]
newvars = obj[1][0:4]
newgrads = obj[1][4:8]

In [ ]:
for epoch in range(0,epoch_num):
    for _ in range(0, total, N):
        X_batch, Y_batch = mnist.train.next_batch(N)
        # TensorFlow method gives the label data in a one hot vetor format. We convert that into a single label.
        Y_batch = np.argmax(Y_batch,axis=1)
        info_dict = inference.update(feed_dict={x: X_batch, y_ph: Y_batch})
        
    train_lossadam[epoch] = info_dict['loss']
    inference.print_progress(info_dict)

In [ ]:
adam = train_lossadam[0:100]
#save loss
with open('adamloss', 'wb') as fp:
    pickle.dump(adam, fp)

In [ ]:
display_draws(train_lossadam, X_batch, epoch_num)

# MISSO